In [1]:
import sys
sys.path.append('/home/watopedia/github_projects/watopedia/')
webserver_files = '/home/watopedia/Desktop/TEMP/cloud_ml_webrtc/webserver/'
import logging
import tornado.escape
import tornado.ioloop
import tornado.options
import tornado.web
import tornado.websocket
import os.path
import uuid
from PIL import Image
import time

#from io import StringIO
from io import BytesIO
import uuid
import numpy
import json
from tornado.options import define, options
import tornado.httpserver
import numpy as np
import ssl
from tornado.ioloop import PeriodicCallback
import base64
port = 8080
import threading

In [2]:
# Add the utlities to the path
import sys
sys.path.append('/home/watopedia/github_projects/watopedia/')


import matplotlib.pyplot as plt
%matplotlib inline
import aipodMain.utilities.constants as constants

import warnings
warnings.filterwarnings('ignore')
# Input Source Thread
from aipodMain.threads.ImageInput.AbstractImageInputThread import AbstractImageInputThread

# Predictor Threads
from aipodMain.threads.Predictor.PredictorImage import PredictorImage
from aipodMain.threads.Predictor.VisualizerPredictor import VisualizerPredictor
from aipodMain.threads.Predictor.SubsetImgPredictor2 import SubsetImgPredictor
from aipodMain.threads.Predictor.DarkNet.PredictorDarknet import DarknetYOLO
from aipodMain.utilities.ThreadData import ThreadData
from aipodMain.threads.Predictor.FaceRecognition.Pipeline.Wrapper import FaceDetectionRecognition as FaceNetRecog
# GUI
from aipodMain.GUI.draw_face_frame import draw_face_frame
from aipodMain.GUI.draw_lidar_frame import draw_lidar_frame
from aipodMain.GUI.draw_label_frame import draw_label_frame
from aipodMain.GUI.draw_image_frame import draw_image_frame
from aipodMain.GUI.MAIN_GUI import nn_draw
# Controller
from aipodMain.GUI.CONTROLLER import CONTROLLER

# Helper Functions
from aipodMain.utilities.ai_helper.objectcounting import objectcounting
from aipodMain.utilities.ai_helper import helper as ai_helper
import time
import cv2
from PIL import Image

# Import ROS
from aipodMain.utilities.other.ros.start_ros_processes import start_ros_processes
from aipodMain.threads.ImageInput.ROSImagePublisher import ROSImagePublisher # Publish to a channel
from aipodMain.utilities.other.ros.pointcloud_listener import LSD_SLAM_TOPIC_LISTENER
print("Finished Loading Imports")

Finished Loading Imports


# PARAMETERS

In [3]:
# TRACKER
MIN_HITS = 1 
MAX_AGE = 4

# MODEL SCORE THRESHOLD VALUES
THRESHOLD_Y9K = 0.5
THRESHOLD_YOLO_COCO  = 0.4
THRESHOLD_COCO = 0.4
THRESHOLD_INCEPTION = 0.4
THRESHOLD_WEAPONS_ONLY = 0.6
THRESHOLD_WEAPONS_ONLY_SUB = 0.1
THRESHOLD_WEAPON_PARTS = 0.4
THRESHOLD_WEAPON_PARTS_SUB = 0.4
# Label List
DISP_LABEL_LIST = [
    # Weapons
    'gun-barrel','machine-guns','trigger','handguns', 'rifle',
    # Coco Module
    'person', 'backpack', 'bottle', 'knife', 
    'cell phone', 'scissors', 'Weapon',
    # Faster R-CNN Module
    'Person','Rifle', 'Handgun', 'Shotgun', 'Digital clock', 'Alarm clock', 'Syringe',
      "Bottle", "Knife", "Backpack",
      "Mobile phone"]
# Exclude All Of These Classes
ENABLE_EXCLUDE_LABEL_LIST = True
EXCLUDE_LABEL_LIST = [
    # Faster R-CNN
    'Clothing', 'Face', 'Hair', 'Head', 'Arm', 'Hand', 'Beard', 'bed','sofa',
    # COCO
    "mouse" ,
    # Weapons
    'NA',
    # YOLO 9K
    'Weapon','y9k-Rifle','y9k-Handgun','y9k-Shotgun', 'worker','creation','craftsman','living thing',
    'instrumentality', 'artifact', 'organism', 'person-9k', 'matter', 'expert' ,'whole', 'entertainer', 'nutriment',
    'backpack-9k', 'African','Mexican','Arabian'
]

# Custom Image Data

In [4]:
from collections import deque
import numpy as np
import time

class CustomImageThread(AbstractImageInputThread):
    def __init__(self, IMAGE_WIDTH = 640 ,IMAGE_HEIGHT = 480):
        name = 'Custom Image Thread'
        super().__init__(name, IMAGE_WIDTH ,IMAGE_HEIGHT)
        empty_image = np.zeros(shape=(20,20,3))
        self.queue = deque([empty_image])
        self.image_data.image_np = empty_image
        self.image_data.isInit = True

    def updateImg(self, threadName):
        while not self.done:
            last_image = self.image_data.image_np
            try:
                last_image = self.queue.popleft()[:,:,::-1]
            except:
                pass
            self.image_data.image_np = last_image
            time.sleep(0.05)
            
    def appendElement(self, image_np):
        '''
        Adds an image for processing to the queue.
        '''
        if len(self.queue) < 10:
            self.queue.append(image_np)
        else:
            self.queue.popleft() # Remove the top most element.

In [5]:
objCounting = objectcounting()
thread_image = CustomImageThread()
thread_image.start()
image_data = thread_image.image_data
START_PREDICTOR = True
thread_data = ThreadData()

Starting Custom Image Thread


# Load Predictor Threads

In [6]:

detector_thresh = 0.8
recognition_thresh = 0.5
DETECTOR_IMG_SCALE = 0.5
TRACKER = 'Legacy'
thread_frm = FaceNetRecog(thread_image.image_data,
                  '/home/watopedia/github_projects/aipod-data/face_recognition/train_datasets/watopedia-2018-10-13/svm_models/facenet_mtcnn_align/classifier.pkl',
                             detector_thresh=detector_thresh,
                             recognition_thresh=recognition_thresh,
                             detection_tracker=TRACKER,
                             DETECTOR_IMG_SCALE = DETECTOR_IMG_SCALE)

thread_data.addThreadElement(thread_frm,'f','FRM')




Loading MTCNN Model
MTCNN. Setting parameters.
Loading Model File: /home/watopedia/github_projects/watopedia/aipodMain/nnModels/faceRecognition/faceNet/20180408-102900/20180408-102900.pb
Finished Loading Model. Allocated 20.0% of GPU.
FaceNetModel. Resize Image to 160x160
Starting MTCNN Face Predictor
Starting Face Recognition Thread.


In [7]:
thread_gsm_1 = DarknetYOLO("YOLO COCO",image_data,
                        THRESHOLD_YOLO_COCO,
                         TRACKER_TYPE='Legacy',
                        YOLO_DIR="/home/watopedia/github_projects/watopedia/nnModels/yolov3/coco/608/",
                          IMG_SCALE=0.5)

Finished creating labelmap. Saved in /home/watopedia/github_projects/watopedia/nnModels/yolov3/coco/608//label_map.pbtxt


In [8]:
thread_gsm_2 = DarknetYOLO("YOLO 9K",image_data,
                        THRESHOLD_YOLO_COCO,
                         TRACKER_TYPE='Legacy',
                        YOLO_DIR="/home/watopedia/github_projects/watopedia/nnModels/yolov3/y9k/",
                          IMG_SCALE=0.5)

thread_data.addThreadElement(thread_gsm_1,'y','GSM-C',always_on=False, ignore_thread=False)
thread_data.addThreadElement(thread_gsm_2,'u','GSM-9K',always_on=False, ignore_thread=False)


COCO_WEAPON_PARTS_ONLY = 0.2
v4_coco_wp = PredictorImage('weapon parts only v4',
                       '/home/watopedia/github_projects/aipod-data/weapons/2018-10-26/model/weapon_parts_only-vB/ssd_300/frozen/frozen_inference_graph.pb',
                       '/home/watopedia/github_projects/aipod-data/weapons/2018-10-26/model/weapon_parts_only-vB/ssd_300/label_map.pbtxt',
                       image_data,
                       score_thresh=COCO_WEAPON_PARTS_ONLY,
                       TRACKER_TYPE = 'Legacy',
                        IMG_SCALE=0.5)
thread_data.addThreadElement(v4_coco_wp,'s','Weapon Parts Only',always_on=False, ignore_thread=False)

#################################
COCO_WEAPON_ONLY = 0.3
v5_coco_wo = PredictorImage('weapon only v5',
                       '/home/watopedia/github_projects/aipod-data/weapons/2018-10-26/model/weapons_only-vA/ssd_300/frozen/frozen_inference_graph.pb',
                       '/home/watopedia/github_projects/aipod-data/weapons/2018-10-26/model/weapons_only-vA/ssd_300/label_map.pbtxt',
                       image_data,
                       score_thresh=COCO_WEAPON_ONLY,
                       TRACKER_TYPE = 'Legacy',
                           IMG_SCALE=1.0)
thread_data.addThreadElement(v5_coco_wo,'w','Weapons Only',always_on=False, ignore_thread=False)





thread_vis = VisualizerPredictor("Visualization Thread VISUALIZATION",                               
                                 constants.FACE_DETECTOR_SSD_MODEL,
                                     image_data, MOBILE_VIS_LAYERS = True)


thread_data.start_all()
print("Done")

Finished creating labelmap. Saved in /home/watopedia/github_projects/watopedia/nnModels/yolov3/y9k//label_map.pbtxt
Loading Model File: /home/watopedia/github_projects/aipod-data/weapons/2018-10-26/model/weapon_parts_only-vB/ssd_300/frozen/frozen_inference_graph.pb
Finished Loading Model
Loading Model File: /home/watopedia/github_projects/aipod-data/weapons/2018-10-26/model/weapons_only-vA/ssd_300/frozen/frozen_inference_graph.pb
Finished Loading Model
Tracker Disabled.
helper_nn_models_utils. get_label_map(). Warning. No Label Map Defined
Loading Model File: /home/watopedia/github_projects/watopedia/aipodMain/nnModels/faceRecognition/ssd_face_detector/ssd_inference_graph.pb
Alocating 5.0% of GPU
Finished Loading Model
Starting Visualization Thread VISUALIZATION
Starting YOLO COCO
Starting YOLO 9K
Starting weapon parts only v4
Starting weapon only v5
Done


In [9]:
# Neural Network
nnDraw = nn_draw(thread_vis.output_data,640,480,update_interval=5)
t1 = threading.Thread(target=nnDraw.updateWithThread).start()

Loading the Neural Net Layer Drawer


# TORNADO DATA

In [10]:
class Application(tornado.web.Application):
    def __init__(self):
        handlers = [
            (r"/", MainHandler),
            (r"/test", TestHandler),
            (r"/webcam", WebcamHandler),
            (r"/cnn_data", CNNHandler),
            (r"/mlresults", MLResultHandler)
            ]

        settings = dict(
            cookie_secret="asdsafl.rleknknfkjqweonrkbknoijsdfckjnk 234jn",
            template_path=os.path.join(os.path.dirname(webserver_files), "templates"),
            static_path=os.path.join(os.path.dirname(webserver_files), "static"),
            xsrf_cookies=False,
            autoescape=None,
            debug=True
            )
        tornado.web.Application.__init__(self, handlers, **settings)
    def startTornado(self):
        print("Starting Tornaado")
        http_server = tornado.httpserver.HTTPServer(app, ssl_options=ssl_ctx)
        http_server.listen(port)
        tornado.ioloop.IOLoop.current().start()
    def stopTornado(self):
        print("Stopping")
        tornado.ioloop.IOLoop.current().stop()

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("facedetect.html")
        
class TestHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("dashboard.html")
        
class WebcamHandler(tornado.websocket.WebSocketHandler):
    '''
    When data is received, append it to our thread image queue.
    '''
    def open(self):
        print('new webcam connection')
        logging.info('new webcam connection')
        #self.set_nodelay(True)
        data = {}
        data['name'] = 'empty_message'
        self.callback = PeriodicCallback(self.update, 500)
        self.callback.start()
        self.empty_message = json.dumps(data)
        self.t1 = time.time()

    def on_message(self, message):
        image = Image.open(BytesIO(message))
        cv_image = numpy.array(image)
        thread_image.appendElement(cv_image)
        # Send Empty data back
        self.write_message(self.empty_message)
        self.t1 = time.time()  


    def on_close(self):
        logging.info('connection webcam closed')
        print("Connection webcam closed")
        self.callback.stop()
    def update(self):
        # If we haven't heard back, send a wakeup message.
        elapsedTime = time.time() - self.t1
        if elapsedTime > 5:
            self.write_message(self.empty_message)
        
        
class CNNHandler(tornado.websocket.WebSocketHandler):
    '''
    When data is received, append it to our thread image queue.
    '''
    def open(self):
        print('new ccn connection')
        logging.info('new CNN connection')
        self.image_np = np.ones(shape=(6,6,3))
        self.next_message = True
        self.t1 = time.time()
        self.callback = PeriodicCallback(self.update, 200)
        self.callback.start()

    def on_message(self, message):
        self.next_message = True
    def update(self):
        elapsedTime = time.time() - self.t1
        if self.next_message or (elapsedTime > 2):
            image_np = nnDraw.showNeuralNetVisualization()*255.0
            image_np = image_np.astype('uint8')
            data = Image.fromarray(image_np)
            fp = BytesIO()
            data.save(fp, 'JPEG')
            data = base64.b64encode(fp.getvalue())
            data = "data:image/png;base64," + data.decode('utf-8')
            self.write_message(json.dumps({
                "img": data,
                "desc": "img_description",
            }))
            self.next_message = False
            self.t1 = time.time()
        


    def on_close(self):
        logging.info('connection CNN closed')
        self.callback.stop()
        print("Connection CNN closed")

class MLResultHandler (tornado.websocket.WebSocketHandler):
    '''
    When data is received, append it to our thread image queue.
    '''
    def open(self):
        logging.info('new ml result connection')
        self.callback = PeriodicCallback(self.update, 500)
        self.callback.start()
        self.previous_data = []
        self.image_height = 0
        self.image_width = 0
    def on_message(self, message):
        if self.image_height == 0:
            data = json.loads(message)
            try:
                self.image_height = int(data['image_properties']['height'])
                self.image_width = int(data['image_properties']['width'])
                print("received" + " height: " + str(self.image_height) + " width: " + str(self.image_width))
            except:
                print("nope")

    def create_thread_data(self,thread_data):
        all_data = []

        for thread_ in thread_data.thread_list:
            h = self.image_height; w = self.image_width;
            data = {}
            data['type'] = 'thread_data'
            data['name'] = thread_.name
            if 'Face' in thread_.name:
                continue
            data['bbs'] = self.fix_bb_coords(thread_.output_data.bbs.copy(),
                                        h,w)
            data['scores'] = thread_.output_data.scores.tolist()
            class_names = []
            for c in thread_.output_data.classes:
                class_names.append(thread_.output_data.category_index.get(c)['name'])
            data['classes'] = class_names
            data['unique_labels'] = objCounting.countFromList(class_names)
            all_data.append(data)
        return all_data
    def create_face_data(self,output_data):
        data = {}
        data['type'] = 'face_data'
        data['db_image'] = 'omar'
        
    def update(self):
        # First we need to concatenate the output data
        all_data = self.create_thread_data(thread_data)

        # Add the face data. TODO: Pass in the face data!
        data = json.dumps(self.create_face_data(thread_frm.output_data))
        #all_data.append(data)
        # Prevent sending duplicate data.
        if all_data != self.previous_data:
            self.write_message(json.dumps(all_data))
        self.previous_data = all_data

    def fix_bb_coords(self,bbs,h,w):
        for indx, bb in enumerate(bbs):
            bbs[indx][0] = int(bbs[indx][0]*h)
            bbs[indx][1] = int(bbs[indx][1]*w)
            bbs[indx][2] = int(bbs[indx][2]*h)
            bbs[indx][3] = int(bbs[indx][3]*w)  
            
            bbs[indx][0] = max(min(bbs[indx][0],h),0)
            bbs[indx][2] = max(min(bbs[indx][2],h),0)
            bbs[indx][1] = max(min(bbs[indx][1],w),0)
            bbs[indx][3] = max(min(bbs[indx][3],w),0)
                
        return bbs.tolist()

  


    def on_close(self):
        logging.info('connection ml closed')
        self.callback.stop()


In [11]:
thread_gsm_1.continue_predictor()

In [12]:
app = Application()
ssl_ctx = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
ssl_ctx.load_cert_chain(webserver_files + "ssl/domain.crt",
                       webserver_files + "ssl/domain.key")

In [13]:
app.startTornado()

Starting Tornaado


RuntimeError: IOLoop is already running

new webcam connection
new ccn connection
received height: 357 width: 674
